In [31]:
import json
import random

class Character:
    def __init__(self):
        self.faction = ""

        self.schadenslimit = 0
        self.block = 0
        self.equipment = []
        self.faction_notes = ""
        self.chosen_path = ""
        self.selected_options = []

        self.styles = []
        self.values = []
        self.cliffhanger = []
        self.gender = ""
        
        self.chosen_styles = []
        self.assigned_values = []
        self.character_styles = []
        self.character_cliffhangers = []  # List for selected cliffhangers
        #self.select_styles = []
        self.faction_data = self.load_faction_data()
    ##############
    
    # Json File
    def load_faction_data(self):
        with open("faction_data.json", "r", encoding="utf-8") as file:
            return json.load(file)

    ##############    
    # Select Factions
    def choose_faction(self):
        factions = list(self.faction_data.keys()) + ["Randomize Faction"]

        print("Choose your faction:")
        for i, faction in enumerate(factions, 1):
            print(f"{i}. {faction}")

        while True:
            try:
                choice = int(input("Enter the number of your choice: "))
                if 1 <= choice <= len(factions):
                    if factions[choice - 1] == "Randomize Faction":
                        self.random_faction_selection(factions[:-1])
                    else:
                        self.faction = factions[choice - 1]
                        print(f"You chose the {self.faction}!\n")
                        self.apply_faction_specifics()
                    break
                else:
                    print("Please choose a valid number from the list.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    ########### Faction specific stuff
    def faction_specific_styles(self):
        # Special gender handling for Borstenmütter
        # Needed for handling their general styles
        if self.faction == "Borstenmütter":
            self.choose_gender()

        # Load styles and values for the chosen faction
        if self.faction in self.faction_data:
            # Handle Borstenmütter (gender-specific styles)
            if self.faction == "Borstenmütter" and self.gender:
                base_styles = self.faction_data[self.faction]["base_styles"][self.gender]["styles"]
                base_values = self.faction_data[self.faction]["base_styles"][self.gender]["values"]

            # For all other factions
            else:
                base_styles = self.faction_data[self.faction]["base_styles"]["styles"]
                base_values = self.faction_data[self.faction]["base_styles"]["values"]

            # Load additional categories like habit_styles and joker_styles
            habit_styles = self.faction_data[self.faction]["habit_styles"]["styles"]
            habit_values = self.faction_data[self.faction]["habit_styles"]["values"]

            joker_styles = self.faction_data[self.faction]["joker_styles"]["styles"]
            joker_values = self.faction_data[self.faction]["joker_styles"]["values"]

            # Merge all styles and values into one list
            self.styles = base_styles + habit_styles + joker_styles
            self.values = base_values + habit_values + joker_values
            

        
        else:
            print(f"No specific styles defined for {self.faction}.")

    def choose_gender(self): # Borstenmütter
        print("Borstenmütter: Bist du ein Keiler oder Bache?")
        while True:
            gender_input = input("Gib 'Keiler' oder 'Bache' ein: ").lower()
            if gender_input in ["Keiler", "Bache"]:
                self.gender = gender_input
                print(f"You chose {self.gender} Borstenmütter!\n")
                break
            else:
                print("Invalid input. Please enter 'Keiler' or 'Bache'.")

    ############## Architekten
    def apply_faction_specifics(self):
        # General setup for faction styles, equipment, and schadenslimit
        if self.faction in self.faction_data:
            faction_info = self.faction_data[self.faction]
            self.styles = faction_info["base_styles"]["styles"]
            self.character_styles += list(zip(faction_info["base_styles"]["styles"], faction_info["base_styles"]["values"]))
        
        # Architekten-specific logic
        if self.faction == "Architekten":
            self.character_styles.append(("Nanoschwarm", 12))
            self.character_styles.append(("Diplomatie mit Geschenken", 10))
            self.faction_notes = "Solange dein Nanoschwarm aktiv ist, kannst du nicht von Verderbnis betroffen werden."
            self.equipment = "Du bekommst alle Ausrüstung von Seite 420"
            self.schadenslimit = 6

            # Choose Fatalisten or Utopisten path
            self.choose_architekten_path()

    def choose_architekten_path(self):
        print("\nChoose your path for Architekten:")
        print("1. Fatalisten: Wir wollen die Welt von Oni und Tsukuyumi befreien.")
        print("2. Utopisten: Wir wollen die Erde wieder aufbauen und bewohnbar machen.")
        
        while True:
            try:
                choice = int(input("Enter 1 or 2: "))
                if choice == 1:
                    self.chosen_path = "Fatalisten"
                    self.select_architekten_list("fatalists_list")
                    break
                elif choice == 2:
                    self.chosen_path = "Utopisten"
                    self.select_architekten_list("utopists_list")
                    break
                else:
                    print("Please enter a valid number (1 or 2).")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def select_architekten_list(self, path_type):
        # Fatalists or Utopists list, provided as options for selection
        if path_type == "fatalists_list":
            options = [
                "Ich darf keine Tiere verletzen.",
                "Ich muss Menschen in Not unter allen Umständen helfen."
                "Ich darf schwächeren Menschen kein Leid antun, auch nicht durch Unterlassung einer Handlung.",
                "Ich muss jeden Außenstehenden töten, der von der Lage von Engine 0-1 Bescheid weiß, auch wenn das einem anderen Grundsatz widerspricht",
                "Ich muss Oni und Tsukuyumi immer aufhalten oder schwächen, wenn ich kann."
            ]
        elif path_type == "utopists_list":
            options = [
                "Ich darf nur Oni verletzen",
                "Ich muss die Oni und Tsukuyumi immer aufhalten oder schwächen, wenn ich kann.",
                "Ich muss Menschen in Not unter allen Umständen helfen.",
                "Ich darf schwächeren Menschen kein Leid antun, auch nicht durch Unterlassung einer Handlung.",
                "Ich muss die Verderbnis auslöschen, wann immer ich kann.",
                "Ich darf weder Erde noch Wasser verschmutzen"
            ]
        
        print("\nChoose 3 options from the list below:")
        for i, option in enumerate(options, 1):
            print(f"{i}. {option}")

        self.selected_options = []
        while len(self.selected_options) < 3:
            try:
                choice = int(input(f"Choose an option (1-{len(options)}): ")) - 1
                if 0 <= choice < len(options) and options[choice] not in self.selected_options:
                    self.selected_options.append(options[choice])
                    print(f"Selected: {options[choice]}")
                else:
                    print("Invalid choice or already selected.")
            except ValueError:
                print("Invalid input. Please enter a number.")
        
        # Save selected options (you could store them somewhere or display them in a summary later)
        print("\nYou selected the following options:")
        for option in self.selected_options:
            print(f"\n- {option}")


    ############## All Factions share this basically
    def select_styles(self, style_category, count):
        """Select a given number of styles and assign values to them, displaying available options only once."""
        available_styles = self.faction_data[self.faction][style_category]["styles"]
        available_values = self.faction_data[self.faction][style_category]["values"]

        selected_styles = []
        selected_values = []

        # Display available styles only once
        print(f"\nWähle {len(available_values)} {style_category} mit den Werten {available_values}:")
        for i, style in enumerate(available_styles):
            print(f"{i + 1}. {style}")

        while len(selected_styles) < count:
            try:
                choice = int(input(f"\nChoose a {style_category} (1-{len(available_styles)}): ")) - 1
                if 0 <= choice < len(available_styles):
                    selected_style = available_styles[choice]
                    if selected_style not in selected_styles:
                        selected_styles.append(selected_style)
                        selected_values.append(self.assigned_values.pop(0))  # Assign the next available value
                        print(f"Selected: {selected_style}")
                    else:
                        print("You already selected this style.")
                else:
                    print("Invalid choice. Please choose a valid option.")
            except ValueError:
                print("Invalid input. Please enter a number.")

        # Add selected styles and their assigned values as (style, value) tuples to character_styles
        self.character_styles.extend(zip(selected_styles, selected_values))

    def select_cliffhangers(self, count):
        """Select a given number of cliffhangers without assigning values, displaying available options only once."""
        available_cliffhangers = self.faction_data[self.faction]["cliffhanger"]
        selected_cliffhangers = []

        # Display available cliffhangers only once
        print("\n Wähle 2 Cliffhanger:")
        for i, cliffhanger in enumerate(available_cliffhangers):
            print(f"{i + 1}. {cliffhanger}")

        while len(selected_cliffhangers) < count:
            try:
                choice = int(input(f"\nChoose a cliffhanger (1-{len(available_cliffhangers)}): ")) - 1
                if 0 <= choice < len(available_cliffhangers):
                    selected_cliffhanger = available_cliffhangers[choice]
                    if selected_cliffhanger not in selected_cliffhangers:
                        selected_cliffhangers.append(selected_cliffhanger)
                        print(f"Selected: {selected_cliffhanger}")
                    else:
                        print("You already selected this cliffhanger.")
                else:
                    print("Invalid choice. Please choose a valid option.")
            except ValueError:
                print("Invalid input. Please enter a number.")

        # Add selected cliffhangers to character_cliffhangers
        self.character_cliffhangers.extend(selected_cliffhangers)

    def choose_all_styles(self):
        """Handle three different style choices with corresponding values."""
        # Copy the values to be assigned for selection
        self.assigned_values = self.values[:]

        # Base styles selection
        base_style_count = len(self.faction_data[self.faction]["base_styles"]["values"])
        self.select_styles("base_styles", base_style_count)

        # Habit styles selection
        habit_style_count = len(self.faction_data[self.faction]["habit_styles"]["values"])
        self.select_styles("habit_styles", habit_style_count)

        # Joker styles selection
        joker_style_count = len(self.faction_data[self.faction]["joker_styles"]["values"])
        self.select_styles("joker_styles", joker_style_count)

        # Cliffhanger selection (without values)
        cliffhanger_count = 2  # You can change this number based on how many cliffhangers you want the user to choose
        self.select_cliffhangers(cliffhanger_count)

        self.display_summary()


    ########## Display character
    def display_summary(self):
        print(f"\nSummary for faction {self.faction}:")

        # Sort character_styles by the second element in each tuple (value), in descending order
        print("\nStile:")
        sorted_styles = sorted(self.character_styles, key=lambda x: x[1], reverse=True)
        for style, value in sorted_styles:
            print(f"{value} {style}")

        print("\nSelected Cliffhangers:")
        for cliffhanger in self.character_cliffhangers:
            print(f"- {cliffhanger}")

        print(f"\nSchadenslimit: {self.schadenslimit}")
        print(f"Block:")
        print(f"Ausrüstung: {self.equipment}")
        print(f"\nNotes: {self.faction_notes}")
        print(f"Path: {self.chosen_path}")
        print(f"Selected Options: {', '.join(self.selected_options)}")


In [32]:
# Main execution
def main():
    character = Character()
    character.choose_faction()
    character.faction_specific_styles()
    character.choose_all_styles()

if __name__ == "__main__":
    main()

Choose your faction:
1. Architekten
2. Borstenmütter
3. Chronomaster
4. Cybersamurai
5. Corporat
6. Herren der Verlorenen See
7. Krieger von Valhalla
8. Kinder des Löwen
9. Mardukorden
10. Moon Circus
11. Nomads
12. Pan Doa
13. Schwesternschaft der 7
14. Der Schwarm
15. Überlebender
16. Union
17. Wanderer
18. Randomize Faction


Enter the number of your choice:  1


You chose the Architekten!


Choose your path for Architekten:
1. Fatalisten: Wir wollen die Welt von Oni und Tsukuyumi befreien.
2. Utopisten: Wir wollen die Erde wieder aufbauen und bewohnbar machen.


Enter 1 or 2:  1



Choose 3 options from the list below:
1. Ich darf keine Tiere verletzen.
2. Ich muss Menschen in Not unter allen Umständen helfen.Ich darf schwächeren Menschen kein Leid antun, auch nicht durch Unterlassung einer Handlung.
3. Ich muss jeden Außenstehenden töten, der von der Lage von Engine 0-1 Bescheid weiß, auch wenn das einem anderen Grundsatz widerspricht
4. Ich muss Oni und Tsukuyumi immer aufhalten oder schwächen, wenn ich kann.


...

Selected: Unverschämtes Glück*

 Wähle 2 Cliffhanger:
1. Ich formiere den Nanoschwarm neu
2. Ich reinige die Erde oder eine Wasserstelle mit meinem Nanoschwarm
3. Ich sage meine Grundsätze auf
4. Ich ziehe mich alleine zurück
5. Ich erzähle einen Witz



Choose a cliffhanger (1-5):  1


Selected: Ich formiere den Nanoschwarm neu



Choose a cliffhanger (1-5):  2


Selected: Ich reinige die Erde oder eine Wasserstelle mit meinem Nanoschwarm

Summary for faction Architekten:

Stile:
12 Nanoschwarm
12 Feiges Vorgehen-
10 Diplomatie mit Geschenken
6 Unbeholfener Nahkampf
6 Überlegene Ausstrahlung
6 Unbeholfener Nahkampf
6 Überlegene Ausstrahlung
6 Unverschämtes Glück*

Selected Cliffhangers:
- Ich formiere den Nanoschwarm neu
- Ich reinige die Erde oder eine Wasserstelle mit meinem Nanoschwarm

Schadenslimit: 6
Ausrüstung: Du bekommst alle Ausrüstung von Seite 420
Notes: Solange dein Nanoschwarm aktiv ist, kannst du nicht von Verderbnis betroffen werden.
Path: Fatalisten
Selected Options: Ich darf keine Tiere verletzen., Ich muss Menschen in Not unter allen Umständen helfen.Ich darf schwächeren Menschen kein Leid antun, auch nicht durch Unterlassung einer Handlung., Ich muss jeden Außenstehenden töten, der von der Lage von Engine 0-1 Bescheid weiß, auch wenn das einem anderen Grundsatz widerspricht
